In [97]:
import requests 
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
load_dotenv()

True

In [111]:
api_key = os.getenv('API_KEY')
url_daily = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=api_key&datatype=json'
url_weekly = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol=IBM&apikey=api_key&datatype=json'
url_monthly = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=IBM&apikey=api_key&datatype=json'
response_daily = requests.get(url_daily)
response_weekly = requests.get(url_weekly)
response_monthly = requests.get(url_monthly)
data_daily = response_daily.json()
data_weekly = response_weekly.json()
data_monthly = response_monthly.json()
data_weekly

{'Information': 'We have detected your API key as api_key and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [112]:
#Extracing the time series data from the JSON
ts_daily= data_daily['Time Series (Daily)']
ts_weekly= data_weekly['Weekly Time Series']
ts_monthly= data_monthly['Monthly Series']
#converting to DataFrame with dates in the rows
ts_daily_df = pd.DataFrame.from_dict(ts_daily, orient='index')
ts_weekly_df = pd.DataFrame.from_dict(ts_weekly, orient='index')
ts_monthly_df = pd.DataFrame.from_dict(ts_monthly, orient='index')

#converting the index to datetime and cleaning up the columns
ts_daily_df.index = pd.to_datetime(ts_daily_df.index)
ts_daily_df.columns = [col.split('. ')[1]for col in ts_daily_df.columns]

ts_weekly_df.index = pd.to_datetime(ts_weekly_df.index)
ts_weekly_df.columns = [col.split('. ')[1]for col in ts_weekly_df.columns]

ts_monthly_df.index = pd.to_datetime(ts_monthly_df.index)
ts_monthly_df.columns = [col.split('. ')[1]for col in ts_monthly_df.columns]


#changing all the values to numeric
df_daily = ts_daily_df.apply(pd.to_numeric)
df_monthly = ts_monthly_df.apply(pd.to_numeric)
df_weekly = ts_weekly_df.apply(pd.to_numeric)

#changing the index into a column
df_daily = df.reset_index()
df_daily.rename(columns={'index': 'date'}, inplace=True)

df_weekly = df.reset_index()
df_weekly.rename(columns={'index': 'date'}, inplace=True)

df_monthly = df.reset_index()
df_monthly.rename(columns={'index': 'date'}, inplace=True)


KeyError: 'Time Series (Daily)'

In [ ]:
#getting credentials of the database
database = os.getenv('database')
port = os.getenv('port')
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')


#creating an sqlalchemy engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

#loading the daily data into the database
try:
    df_daily.to_sql('Daily_prices', engine ,if_exists ='replace')
    print('Data loaded successfully into the database.')
except Exception as e:
    print('Failed to load daily data into the database.')
    print('Error:',e)

#Loading Weekly data into the database
try:
    df_weekly.to_sql('Weekly_prices', engine ,if_exists ='replace')
    print('Weekly Data loaded successfully into the database.')
except Exception as e:
    print('Failed to load weekly data into the database.')
    print('Error:',e)

#loading monthly data into the database
try:
    df_monthly.to_sql('Monthly_prices', engine ,if_exists ='replace')
    print('Monthly Data loaded successfully into the database.')
except Exception as e:
    print('Failed to load monthly data into the database.')
    print('Error:',e)



Data loaded successfully into the database.
